## **Turi Style Transfer**
Welcome to the style transfer model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

- Load style and content images from S3
- Build a style transfer model.
- Convert the model to CoreML format and save it to the Skafos framework.

The example is based on [Turi Create's Style Transfer example](https://apple.github.io/turicreate/docs/userguide/style_transfer/).

--- 

Execute each cell one-by-one, by selecting the cell and do one of the following:

-  Clicking the "play" button at the top of this frame.
-  Typing 'Control + Enter' or 'Shift + Enter'.

In [ ]:
# If this is your first time in the JupyterLab workspace - install external dependencies
from utilities.dependencies import install
install(timeout=500)

# No need to do this in the future for this notebook

In [ ]:
# Import necessary libraries
import os

import coremltools
import turicreate as tc
from skafossdk import *
from s3fs.core import S3FileSystem

In [ ]:
# Initialize Skafos
ska = Skafos()

# Connect to S3
s3 = S3FileSystem(anon=True)

### 1. **Load the data**
To obtain the data for this example, we performed the following steps:

- We searched for plants (content) and abstract art images (style). 
- The style and content images were manually grabbed from [Pexels.com](https://www.pexels.com/) where you can find free stock images.
- We uploaded the images to S3 so that they could be downloaded here.
- Once pulled, the images were split into a `/contents` folder and `/styles` folder. 
- The images were loaded into 'SFrames' which are Turi Create data structures intended to load and hold data efficiently.

In [ ]:
# Get list of style paths and content paths from S3
style_paths = s3.ls("s3://skafos.example.data/StyleTransferImages/styles/")
content_paths = s3.ls("s3://skafos.example.data/StyleTransferImages/content/")
image_paths = style_paths + content_paths

# If the local paths don't exist, make them
for _dir in ['content', 'styles']:
    if not os.path.exists(_dir):
        os.makedirs(_dir)
        
# Generate two empty image SFrames
styles = tc.SFrame()
content = tc.SFrame()

# Loop over the image paths, adding them to the SFrames
for p in image_paths:
    
    # Distinguish styles from content based on path
    _type = p.split("/")[-2]
    local_path = "/".join(p.split("/")[-2:])
    img1 = s3.get("s3://" + p, "./"+local_path) # download the image
    
    # Add the image to the appropriate SFrame
    if _type == "styles":
        styles = styles.append(tc.load_images(local_path))
    if _type == "content":
        content = content.append(tc.load_images(local_path))

### 2. **Build the model**
We pass the data to the `tc.style_transfer.create` function. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/style_transfer/). In the `tc.style_transfer.create` function below, we set `max_iterations` to 1 for run time concerns. For more more about this, check out our documentation in the `advanced_usage` section of this repo or [Turi Create's documentation](https://apple.github.io/turicreate/docs/userguide/style_transfer/).

**Note: GPU support is coming soon. The code below has been commented out because it will take a long time (~4 days) to train with CPUs. Additionally, if you don't update the training dataset, it will re-create the baseline style transfer model from the integration guide on the Skafos dashboard.**

In [ ]:
# Build the Style Transfer model
#model = tc.style_transfer.create(styles, content, max_iterations=1)

### 3. **Deliver the model**
Once your model has been created, it must be converted to CoreML and saved to the Skafos framework. Once saved, if you wish to push to your iOS devices, you can use the `.deliver()` method below. We've left that commented out for now.

In [ ]:
# Specify the CoreML model name
model_name = 'StyleTransfer'
coreml_model_name = model_name + '.mlmodel'

# Export the trained model to CoreML format
res = model.export_coreml(coreml_model_name) 

## Use coremltools to convert model weights to half-precision.
## This may be necessary if you have memory concerns within your app
#model_spec = coremltools.utils.load_spec(coreml_model_name)
#model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
#coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# Save model asset to Skafos
ska.asset_manager.save(
    name=model_name,              # Name used to load or deliver asset, also used within the Swift SDK.
    files=coreml_model_name,      # File or list of files to bundle together as a versioned asset.
    tags=['latest'],              # User-defined tags to help distinguish your asset.
    access='public'               # Asset access- public/private.
)

In [ ]:
# Deliver asset to devices (push)
#ska.asset_manager.deliver(
#  name=model_name,                # Name used to load or deliver asset, also used within the Swift SDK.
#  tag='latest',                   # User-defined tags to help distinguish your asset.
#  dev=True                        # Push asset through Apple's APNS dev or prod server
#)